In [1]:
%matplotlib inline
import h5py
import numpy as np
import os
from UG2.utils import data as data_utils
from UG2.utils import image as image_utils
from UG2.config import Config
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.autograd import Variable
import scipy.ndimage as im
import time
from UG2 import main

In [2]:
data_path            = "/data/UG2_data/h5Files/"
traindedModels_path  = "/data/UG2_data/trainedModels/"

In [7]:
## create blurred-dataset 
kernel_blurring_flag= True
num_images          = 100
factor              = 1
patch_size          = np.array([64, 64])
epochsList          = [700]
learningRateList    = [0.001]
numberTrainingKernel= [4]
flags               = [0,1]
numberOfExperiments = len(epochsList)*len(learningRateList)
dataset_name        = "BSDBlurred"

In [8]:
trainStartIndex = 0
numberCompletedExperiments = 0
for i in range(trainStartIndex, len(epochsList)):
    for nLR in learningRateList:
        for nTK in numberTrainingKernel:
            config = Config() 
            config.epochs = epochsList[i]
            config.cuda = True
            config.data_parallel = False
            config.lr = nLR
            config.data_path = data_path
            if kernel_blurring_flag:
                config.data_files = [os.path.join(data_path, dataset_name+"_nTK_"+str(nTK)+ "_flag_" + "_".join([str(f) for f in flags])+"factor"+str(factor)+"nImages"+str(num_images)+".h5")]
                print("Loading file: "+str(config.data_files))
            else:        
                config.data_files = [os.path.join(data_path, dataset_name+".h5")]
                print("Loading file: "+str(config.data_files))
            config.batch_size = 50
            config.print_step = 100
            config.checkpoint = 1
            config.exit_loss_window = 5
            config.loss_epsilon = 0.01
            config.resume_training_flag = False
            config.kernel_blurring_flag = False
            config.discriminator = "feat_ext"
            config.factor      = factor
            config.img_size    = 64
            config.num_workers = 8
            config.data_format = "h5"
            if i:
                config.epochs = epochsList[i]-epochsList[i-1]
                config.resume_training_flag = False
                config.resume_model_path = traindedModels_path
                config.resume_model_name = "modellrRate"+str(nLR).replace(".","_")+"nEpochs"+str(epochsList[i-1])+".pth"
            
            config.model_path = traindedModels_path
            config.model_name = "modellrRate"+str(nLR).replace(".","_")+"nEpochs"+str(epochsList[i])+"nTK"+str(nTK)+"factor"+str(factor)+"nImages"+str(num_images)+".pth"
            numberCompletedExperiments = numberCompletedExperiments+1
            main.train(config)
        
            print("Completed "+str(numberCompletedExperiments)+" out of "+str(numberOfExperiments))
        

Loading file: ['/data/UG2_data/h5Files/BSDBlurred_nTK_4_flag_0_1factor1nImages100.h5']
time:  4.39734148979187  Error:  2.1672413891
saved checkpoint at epoch:  0
time:  4.177564382553101  Error:  0.467243267731
saved checkpoint at epoch:  100
time:  4.124566555023193  Error:  0.371595900167
saved checkpoint at epoch:  200
time:  4.399526119232178  Error:  0.325234251944
saved checkpoint at epoch:  300
time:  4.411777019500732  Error:  0.279112363403
saved checkpoint at epoch:  400
time:  4.436954736709595  Error:  0.253594929522
saved checkpoint at epoch:  500
time:  4.2032856941223145  Error:  0.23548098044
saved checkpoint at epoch:  600
Model saved as:  modellrRate0_001nEpochs700nTK4factor1nImages100.pth
Completed 1 out of 1
